In [1]:
#Load all the packages we'll need
import altair as alt
import numpy as np
import pandas as pd
# Couldn't figure out a way to code it in JAGS, so using numpy

In [2]:
def set_up_game(all_doors):
    return np.random.choice(all_doors, 1)

In [3]:
def pick_first(all_doors):
    return np.random.choice(all_doors, 1)

In [4]:
def open_goat(closed_doors, car, my_door):
    can_open = np.logical_and(closed_doors != car, closed_doors != my_door)
    goat = np.random.choice(closed_doors[can_open], 1)
    closed_doors = closed_doors[closed_doors != goat]
    return goat, closed_doors

In [5]:
def switch_or_stay(p, my_door, closed_doors):
    # decision to switch follows Bernoulli(p)
    # if switching, choose another closed door at random
    if np.random.binomial(1, p) == 0:
        return True, my_door
    else:
        return False, np.random.choice(closed_doors[closed_doors != my_door], 1)

In [6]:
def simulate(N, M, p, verbose = False):
    closed_doors = np.arange(N)
    car = pick_first(closed_doors)
    if verbose:
        print(f"Set up game: car at door {car} (shh don't tell anyone)")
    my_door = pick_first(closed_doors)
    if verbose:
        print(f"First pick: picked door {my_door}")
    for i in range(M):
        goat, closed_doors = open_goat(closed_doors, car, my_door)
        if verbose:
            print(f"Turn {i+1}: goat at door {goat}!")
        switched, my_door = switch_or_stay(p, my_door, closed_doors)
        if switched & verbose:
            print(f"Switched to door {my_door}")
    if my_door == car:
        if verbose:
            print(f"\nPicked a car!\n")
        return True
    if verbose:
        print(f"\nPicked a goat :(\n")
    return False

In [7]:
def simulate_many(N, M, p, reps):
    won = 0
    for rep in range(reps):
        won += simulate(N, M, p)
    return won / reps

In [8]:
def simulation_plot(N, M, p, reps):
    data_dict = {}
    for p in np.arange(0, 1.1, 0.1):
        data_dict[p] = simulate_many(N, M, p, reps)
    data = pd.DataFrame.from_dict({'p' : data_dict.keys(),
                                  'success_rate' : data_dict.values()})
    chart = alt.Chart(data).mark_bar(size=35).encode(
        x=alt.X('p', title = "p", axis=alt.Axis(values=list(np.arange(0,1.1,0.1)))),
        y=alt.Y("success_rate", title="success_rate"))
    
    return chart

chart = simulation_plot(3, 1, 1, 2000)        
chart

alt.Chart(...)

In [9]:
# This cell may take a long time to run
chart = simulation_plot(20, 7, 1, 2000)        
chart

alt.Chart(...)